In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in files:
    with open(f) as fopen:
        urls.extend(json.load(fopen))
urls = sorted(list(set(urls)))
len(urls)

948

In [4]:
r = requests.get(urls[-1])
soup = BeautifulSoup(r.content, "lxml")

In [6]:
soup.find('h1').text

'Your Car Based on Your Chinese Zodiac Sign in 2023'

In [7]:
urls[-1]

'https://www.carsome.my/news/item/zodiac-sign-cars'

In [11]:
divs = soup.find_all('div', {'class': 'tdb-block-inner td-fix-index'})
divs = [div.get_text(separator="\n") for div in divs]
divs

['',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 '',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 'Search',
 'Home\nLifestyle\nYour Car Based on Your Chinese Zodiac Sign in 2023',
 'Your Car Based on Your Chinese Zodiac Sign in 2023',
 'By\n \nDevan S',
 '11/01/2023',
 '\n\n\n\n\n',
 'The Chinese Lunar New Year is upon us and that means it’s time for celebration! It’s also the time to welcome new things into your life for an excellent start to the year, so if you’ve been thinking of upgrading your car, now is the perfect time. We’ve come up with a fun list of cars for your Chinese zodiac sign in 2023 to help make your celebrations more joyous. And who knows, it might just help you find the right car.\n\n\nA little fact – the Chinese zodiac signs are a big part of Chinese culture and have been used for over 2000 years to calculate and represent the years in a 12-year cycle of the Chinese calendar. The animals of the zodiac with their unique personalities have significant roles in Chinese stories and fol

In [12]:
!rm -rf page
!mkdir page

In [13]:
for no, url in tqdm(enumerate(urls)):
    filename = os.path.join('page', f'{no}.json')
    if os.path.exists(filename):
        continue
    
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
            
    soup = BeautifulSoup(r.content, "lxml")
    
    try:
        title = soup.find('h1').text
    except:
        title = None
    
    try:
        divs = soup.find_all('div', {'class': 'tdb-block-inner td-fix-index'})
        body = [div.get_text(separator="\n") for div in divs]
    except:
        body = None

    data = {
        'url': url,
        'title': title,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

948it [07:39,  2.06it/s]


In [14]:
files = sorted(glob('page/*.json'))
len(files)

948

In [15]:
with open('data.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
        
        if not len(data):
            continue
        
        fopen_l.write(f'{json.dumps(data)}\n')

100%|███████████████████████████████████████| 948/948 [00:00<00:00, 7292.76it/s]
